<a href="https://colab.research.google.com/github/OVP2023/NLP/blob/main/DZ2_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B8%D1%84%D0%B8%D0%BA%D0%B0%D1%86%D0%B8%D1%8F_%D0%B2_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''Задание: сделать классификацию данных fakenews.
Используя ноутбук занятия и данные fakenews, нужно три раза разными способами получить на задаче
классификации значение f1 выше 0.91 для методов на sklearn и выше 0.52 для методов на pytorch.
Результат: ноутбук с обученной моделью, где показаны результаты метрик качества: accuracy и f1-score
Инструменты:
● ноутбук занятия
● fakenews
● NLTK
● Sklearn
● PyTorch'''

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Datasets/NLP/Constraint_Train.csv')
df.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
'''Метод 1: наивный (векторизатор CountVectorizer плюс модель логистической регрессии)'''

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

In [4]:
vec = CountVectorizer()
bow = vec.fit_transform(df.tweet)
X_train, X_test, y_train, y_test = train_test_split(bow, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.93      0.92      0.92      1025
        real       0.92      0.93      0.93      1094

    accuracy                           0.92      2119
   macro avg       0.93      0.92      0.92      2119
weighted avg       0.92      0.92      0.92      2119



In [ ]:
# Полученная метрика f1-score > 0.91

In [ ]:
'''Метод 2: применим тематическое моделирование для предобработки тестов, получения эмбедингов с помощью Tf-id'''

In [56]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
nltk.download('stopwords')
mystopwords = stopwords.words('english') +['The']

def  remove_stopwords(text, mystopwords=mystopwords):
    try:
        return " ".join([token for token in text.split() if not token in mystopwords])
    except:
        return ""

df['stopwords']= df.tweet.apply(remove_stopwords)
df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,tweet,label,stopwords,lemma
0,1,The CDC currently reports 99031 deaths. In gen...,real,CDC currently reports 99031 deaths. In general...,the cdc currently report 99031 deaths. in gene...
1,2,States reported 1121 deaths a small rise from ...,real,States reported 1121 deaths small rise last Tu...,states reported 1121 death a small rise from l...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,Politically Correct Woman (Almost) Uses Pandem...,politically correct woman (almost) uses pandem...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,#IndiaFightsCorona: We 1524 #COVID testing lab...,#indiafightscorona: we have 1524 #covid testin...
4,5,Populous states can generate large case counts...,real,Populous states generate large case counts loo...,populous state can generate large case count b...


In [23]:
nltk.download('omw-1.4')

lem = WordNetLemmatizer()

def lemmatize(text, mystem=lem):
    try:
        return " ".join([lem.lemmatize(w, pos="n").lower() for w in text.split(' ')]).strip()
    except:
        return " "

df['lemma']= df.tweet.apply(lemmatize)
df.head(10)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


,id,tweet,label,stopwords,lemma
0,1,The CDC currently reports 99031 deaths. In gen...,real,CDC currently reports 99031 deaths. In general...,the cdc currently report 99031 deaths. in gene...
1,2,States reported 1121 deaths a small rise from ...,real,States reported 1121 deaths small rise last Tu...,states reported 1121 death a small rise from l...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,Politically Correct Woman (Almost) Uses Pandem...,politically correct woman (almost) uses pandem...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,#IndiaFightsCorona: We 1524 #COVID testing lab...,#indiafightscorona: we have 1524 #covid testin...
4,5,Populous states can generate large case counts...,real,Populous states generate large case counts loo...,populous state can generate large case count b...
5,6,"Covid Act Now found ""on average each person in...",real,"Covid Act Now found ""on average person Illinoi...","covid act now found ""on average each person in..."
6,7,If you tested positive for #COVID19 and have n...,real,If tested positive #COVID19 symptoms stay home...,if you tested positive for #covid19 and have n...
7,8,Obama Calls Trump’s Coronavirus Response A Cha...,fake,Obama Calls Trump’s Coronavirus Response A Cha...,obama calls trump’s coronavirus response a cha...
8,9,"???Clearly, the Obama administration did not l...",fake,"???Clearly, Obama administration leave kind ga...","???clearly, the obama administration did not l..."
9,10,Retraction—Hydroxychloroquine or chloroquine w...,fake,Retraction—Hydroxychloroquine chloroquine with...,retraction—hydroxychloroquine or chloroquine w...


In [60]:
vec2 = TfidfVectorizer(ngram_range=(1,2), max_features=10000, sublinear_tf=True)

bow2 = vec2.fit_transform(df.lemma)
X_train, X_test, y_train, y_test = train_test_split(bow2, df.label, test_size=0.33)
model = LogisticRegression()
model.fit(X_train, y_train)
predicted = model.predict(X_test)
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

        fake       0.93      0.93      0.93       988
        real       0.94      0.94      0.94      1131

    accuracy                           0.94      2119
   macro avg       0.94      0.94      0.94      2119
weighted avg       0.94      0.94      0.94      2119



In [ ]:
# Полученная метрика f1-score > 0.91, а результат метрики лучше чем в методе 1